In [ ]:
import pandas as pd

from wellbeing_and_machine_learning.config import BLD

In [ ]:
ppathl = pd.read_stata(
    BLD / "data" / "ppathl.dta",
    columns=["pid", "cid", "syear", "gebjahr", "birthregion", "migback", "sex"],
)

In [ ]:
pgen = pd.read_stata(
    BLD / "data" / "pgen.dta",
    columns=["pid", "cid", "syear", "pgbilzeit", "pglfs", "pgfamstd"],
)

In [ ]:
hgen = pd.read_stata(BLD / "data" / "hgen.dta", columns=["cid", "syear", "hghinc"])

In [ ]:
chunksize = 10**6  # adjust this value depending on your system's memory
chunks = []

for chunk in pd.read_stata(
    BLD / "data" / "pl.dta",
    columns=[
        "pid",
        "cid",
        "syear",
        "ple0006",
        "ple0007",
        "plc0446",
        "ple0059",
        "pmonin",
        "plh0258_h",
        "plb0178",
        "plh0182",
    ],
    chunksize=chunksize,
):
    chunks.append(chunk)

pl = pd.concat(chunks, axis=0)

In [ ]:
hbrutto = pd.read_stata(BLD / "data" / "hbrutto.dta", columns=["cid", "syear", "hhgr"])

In [ ]:
hl = pd.read_stata(BLD / "data" / "hl.dta", columns=["cid", "syear", "hlf0001_v3"])

In [ ]:
merge_household = pd.DataFrame()
merge_household = pd.merge(hgen, hbrutto, on=["cid", "syear"], how="outer")

In [ ]:
merge_household = pd.merge(merge_household, hl, on=["cid", "syear"], how="outer")

In [ ]:
merge_household.size

In [ ]:
hl.size

In [ ]:
merge_household.head()

In [ ]:
def merge_household_data(household_data):
    merged_df = household_data[0]
    for df in household_data[1:]:
        merged_df = merged_df.merge(df, on=["cid", "syear"], how="outer")
    return merged_df

In [ ]:
household_data = [hgen, hbrutto, hl]

In [ ]:
test = merge_household_data(household_data)

In [ ]:
test.head()